# Ray RLlib - Extra Application Example - FrozenLake-v0

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

This example uses [RLlib](https://ray.readthedocs.io/en/latest/rllib.html) to train a policy with the `FrozenLake-v0` environment ([gym.openai.com/envs/FrozenLake-v0/](https://gym.openai.com/envs/FrozenLake-v0/)).

For more background about this problem, see:

* ["Introduction to Reinforcement Learning: the Frozen Lake Example"](https://reinforcementlearning4.fun/2019/06/09/introduction-reinforcement-learning-frozen-lake-example/), [Rodolfo Mendes](https://twitter.com/rodmsmendes)
* ["Gym Tutorial: The Frozen Lake"](https://reinforcementlearning4.fun/2019/06/16/gym-tutorial-frozen-lake/), [Rodolfo Mendes](https://twitter.com/rodmsmendes)

In [ ]:
import pandas as pd
import json, os, shutil, sys
import ray
import ray.rllib.agents.ppo as ppo

In [ ]:
ray.init(ignore_reinit_error=True)

In [ ]:
print(f'Dashboard URL: http://{ray.get_webui_url()}')

Set up the checkpoint location:

In [ ]:
checkpoint_root = 'tmp/ppo/frozen-lake'
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

Next we'll train an RLlib policy with the `FrozenLake-v0` environment.

By default, training runs for `10` iterations. Increase the `n_iter` setting if you want to see the resulting rewards improve.
Also note that *checkpoints* get saved after each iteration into the `/tmp/ppo/taxi` directory.

> **Note:** If you prefer to use a different directory root than `/tmp`, change it in the next cell **and** in the `rllib rollout` command below.

In [ ]:
SELECT_ENV = "FrozenLake-v0"
N_ITER = 10

config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"

agent = ppo.PPOTrainer(config, env=SELECT_ENV)

In [ ]:
results = []
episode_data = []
episode_json = []

for n in range(N_ITER):
    result = agent.train()
    results.append(result)
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']}
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = agent.save(checkpoint_root)
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')
reward_history = []

In [ ]:
import pprint

policy = agent.get_policy()
model = policy.model

pprint.pprint(model.variables())
pprint.pprint(model.value_function())

print(model.base_model.summary())

In [ ]:
ray.shutdown()  # "Undo ray.init()".

## Rollout

Next we'll use the [`rollout` script](https://ray.readthedocs.io/en/latest/rllib-training.html#evaluating-trained-policies) to evaluate the trained policy.

The following rollout visualizes the "character" agent operating within its simulation: trying to find a walkable path to a goal tile. 

The [FrozenLake-v0 environment](https://gym.openai.com/envs/FrozenLake-v0/) documentation provides a detailed explanation of the text encoding. The *observation space* is defined as a 4x4 grid:

  * `S` -- starting point, safe
  * `F` -- frozen surface, safe
  * `H` -- hole, fall to your doom
  * `G` -- goal, where the frisbee is located
  * orange rectangle shows where the character/agent is currently located
  
Note that for each action, the grid gets printed first followed by the action.

The *action space* is defined by four possible movements across the grid on the frozen lake:

  * Up
  * Down
  * Left
  * Right

The *rewards* at the end of each episode are structured as:

  * 1 if you reach the goal
  * 0 otherwise

An episode ends when you reach the goal or fall in a hole.

In [ ]:
!rllib rollout \
    tmp/ppo/frozen-lake/checkpoint_10/checkpoint-10 \
    --config "{\"env\": \"FrozenLake-v0\"}" \
    --run PPO \
    --steps 2000

The rollout uses the second saved checkpoint, evaluated through `2000` steps.
Modify the path to view other checkpoints.

## Exercise ("Homework")

In addition to _Taxi_ and _Frozen Lake_, there are other so-called ["toy text"](https://gym.openai.com/envs/#toy_text) problems you can try.